In [1]:
#langchain imports
from langchain.llms import OpenAIChat
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

from langchain.chains import TransformChain, SequentialChain
from langchain import PromptTemplate, LLMChain
import formulas


import pandas as pd
import nest_asyncio
import asyncio
import stripe
import os
import json
import re

from dotenv import load_dotenv
load_dotenv()
key = os.getenv('OPENAI_API_KEY')

#colors for output prints
RED = "\033[91m"
GREEN = "\033[92m"
RESET = "\033[0m"
BOLD = "\033[1m"


chat = OpenAIChat(openai_api_key=key)
gpt4 = OpenAIChat(openai_api_key=key, model_name="gpt-4")
embeddings = OpenAIEmbeddings(openai_api_key=key)

#iterate over the contents of data/services and load them into the vectorstore
service_loaders = []
service_docs = []

#create a list with the names of the files to be loaded in the data/services folder
for file in os.listdir('data/services'):
    if file.endswith('.json'):
        loader = TextLoader('data/services/' + file)
        service_loaders.append(loader)

for loader in service_loaders:
    service_docs.extend(loader.load())

service_embeddings = OpenAIEmbeddings(openai_api_key=key)
service_description_search = Chroma.from_documents(service_docs, service_embeddings)


Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [3]:
#langchain imports
from langchain.llms import OpenAIChat
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import TransformChain, SequentialChain
from langchain import PromptTemplate, LLMChain
import formulas


import pandas as pd
import nest_asyncio
import asyncio
import stripe
import os
import json
import re

from dotenv import load_dotenv
load_dotenv()
key = os.getenv('OPENAI_API_KEY')

#colors for output prints
RED = "\033[91m"
GREEN = "\033[92m"
RESET = "\033[0m"
BOLD = "\033[1m"


chat = OpenAIChat(openai_api_key=key)
gpt4 = OpenAIChat(openai_api_key=key, model_name="gpt-4")
embeddings = OpenAIEmbeddings(openai_api_key=key)

all_services_df = pd.read_csv('data/services_search - Sheet1.csv')
all_services = []
for index, row in all_services_df.iterrows():
    all_services.append(row.to_dict())

#classes to make it easier to sample from the services specific characteristics for embeddings/context window
class Programs:
    def __init__(self, services):
        self.services = services

    def get_service(self, name):
        for service in self.services:
            if service['Name'] == name:
                return service
        return None
    
class Program:
    def __init__(self, dict):
        self.dict = dict

#create a container for a companys program offerings
all_services_list = Programs(all_services)

# Retrieve situational awareness from the vector store
situational_experience = Chroma(persist_directory='data/situation-db', embedding_function=embeddings)

def get_situational_awareness(inputs: dict) -> dict:
    print(GREEN + 'STEP: Getting situational experiences' + RESET)
    conversastion = inputs["conversation_history"]
    knowledge = situational_experience.similarity_search(conversastion, 5)
    knowledge = [message.page_content for message in knowledge]
    print(RED + 'Situational experiences found: ', knowledge, RESET) 
    print(GREEN + 'STEP: Analyzing situational experiences' + RESET)       
    return { "situational_experience": knowledge }

retrieve_situational_awareness = TransformChain(transform=get_situational_awareness, input_variables=["conversation_history"], output_variables=["situational_experience"])

# Create a link that collapses nuggets of business knowledge with the customer's situation
analyze_situation_template = """Your goal is to identify how severe this issue is. Think step by step about your answer and use the information you provided to make you answer. After your analysis, please rate the severity as Low, Medium or High.
Conversation so Far: 
{conversation_history}
Property Type: {property_type}
Below are some possible practical experiences from our technicians that may be useful. Some of it may not be relevant to your answer.
{situational_experience}
----
Your Analysis:
"""

analyyze_situation_prompt = PromptTemplate(template=analyze_situation_template, input_variables=["situational_experience", "property_type", "conversation_history"])

analyze_issue_chain = LLMChain(
    llm=chat, 
    prompt=analyyze_situation_prompt, 
    output_key="situation_analysis"
)

severity_grade_template = """Provide a rating for the severity of the issue. Please rate the severity as "Low", "Medium" or "High".
Issue Analysis: {situation_analysis}
Your Rating:"""

severity_grade_prompt = PromptTemplate(template=severity_grade_template, input_variables=["situation_analysis"])

severity_grade_chain = LLMChain(llm=gpt4, prompt=severity_grade_prompt, output_key="severity_grade_raw")

def get_severity_grade(inputs: dict) -> dict:
    print(GREEN + 'STEP: Getting severity grade' + RESET)
    response = inputs["severity_grade_raw"]
    pattern = re.compile(r'(Low|Medium|High)', re.IGNORECASE)
    matches = pattern.findall(response)

    if matches:
        response = matches[-1]
        severity_grade = response.strip().lower()
        print(RED + 'Severity grade found: ', severity_grade, RESET)
    else:
        severity_grade = None
        print(RED + 'Severity grade not found', RESET)
   
    return { "severity_grade": severity_grade }

get_severity_grade_chain = TransformChain(transform=get_severity_grade, input_variables=["severity_grade_raw"], output_variables=["severity_grade"])

def get_relevant_services(inputs: dict) -> dict:
    analysis = inputs["situation_analysis"]
    print(GREEN + 'STEP: Getting relevant services' + RESET)
    print('Analysis: ', analysis)
    service_search_results = service_description_search.similarity_search(analysis, 3)
    #extract the services as a dict from the search results (they are json stored in .page_content)
    service_search_results = [json.loads(result.page_content) for result in service_search_results]

    print('services found')
    for service in service_search_results:
        print(service['Name'])

    detailed_services = []
    for service in service_search_results:
        service_name = service['Name']
        detailed_service = all_services_list.get_service(service_name)
        detailed_services.append(Program(detailed_service))

    relevant_services_text = ""
    for service in detailed_services:
        relevant_services_text += service.dict['Name'] + ": "
        relevant_services_text += service.dict['Description'] + "\n"
        relevant_services_text += "Best for: " + service.dict['Best for'] + "\n"
        relevant_services_text += "Covers: " + service.dict['Covered Pests'] + "\n"
        relevant_services_text += "Protection: " + service.dict['Level of Protection'] + "\n"
        relevant_services_text += "Price: " + service.dict['Cost'] + "\n\n"
    
    print(RED + 'STEP: Completed getting services: ', detailed_services, RESET)
    print(GREEN + 'STEP: Begin parsing the formulas' + RESET)

    for service in detailed_services:
        recurring_formula = service.dict['Recurring Formula']
        recurring_formula = recurring_formula.replace("{square_footage}", str(inputs["square_footage"]))
        recurring_formula = recurring_formula.replace("{target}", str(inputs["target"]))
        recurring_formula = recurring_formula.replace("{severity}", str(inputs["severity_grade"]))
        recurring_formula = recurring_formula.replace("{acres}", str(inputs["acres"]))
        service.dict['Recurring Formula'] = recurring_formula

        initial_formula = service.dict['Initial Formula']
        initial_formula = initial_formula.replace("{square_footage}", str(inputs["square_footage"]))
        initial_formula = initial_formula.replace("{target}", str(inputs["target"]))
        initial_formula = initial_formula.replace("{severity}", str(inputs["severity_grade"]))
        initial_formula = initial_formula.replace("{acres}", str(inputs["acres"]))
        service.dict['Initial Formula'] = initial_formula
    

    print(GREEN + 'STEP: Completed parsing the formulas' + RESET)

    return { "relevant_services_dict": detailed_services, "relevant_services_text": relevant_services_text }

get_relevant_services_chain = TransformChain(transform=get_relevant_services, input_variables=["situation_analysis"], output_variables=["relevant_services_dict", "relevant_services_text"])


def get_pricing(inputs: dict) -> dict:
    print(GREEN + 'STEP: Generating recurring price' + RESET)
    services = inputs["relevant_services_dict"]

    for service in services:
        recurring_formula = service.dict['Recurring Formula']
        intial_formula = service.dict['Initial Formula']
        
        try:
            recurring_formula_parser = formulas.Parser().ast('=' + recurring_formula)[1].compile()
            recurring_price = recurring_formula_parser()
            recurring_price = str(recurring_price)
        except:
            recurring_price = "Error in Calculating Price"

        try:
            initial_formula_parser = formulas.Parser().ast('=' + intial_formula)[1].compile()
            initial_price = initial_formula_parser()
            initial_price = str(initial_price)
        except:
            initial_price = "Error in Calculating Price"

        service.dict['Recurring Price'] = recurring_price
        service.dict['Initial Price'] = initial_price

    
    service_offers_formatted = ""
    for service in services:
        service_offers_formatted += service.dict['Outward Facing'] + '\n'
        service_offers_formatted += 'Initial Price: $' + service.dict['Initial Price'] + '\n'
        service_offers_formatted += 'Recurring Price: $' + service.dict['Recurring Price'] + '\n'
        service_offers_formatted += 'Description: ' + service.dict['Description'] + '\n'
        service_offers_formatted += 'Covered Pests: ' + service.dict['Covered Pests'] + '\n'
        service_offers_formatted += 'Best For: ' + service.dict['Best for'] + '\n'
        service_offers_formatted += 'Frequency: ' + service.dict['Frequency'] + '\n'
        service_offers_formatted += 'Level of Protection: ' + service.dict['Level of Protection'] + '\n'
        service_offers_formatted += 'Benefits: ' + service.dict['Benefits'] + '\n'
        service_offers_formatted += 'Payment Link: <pay:' + service.dict['Name'] + '>\n'
        service_offers_formatted += '\n'
    return { "complete_services": services, "service_offers_formatted": service_offers_formatted }

calculate_prices_chain = TransformChain(
    transform=get_pricing, 
    input_variables=["relevant_services_dict"], 
    output_variables=["complete_services", "service_offers_formatted"]
)

pitch_template = """You are an expert and friendly salesperson for a pest control company company. You have just finished collecting information from the customer. Provide a helpful and entertaining explaination of the best service you found for them based on the following service and issue. Use informal language. Focus on service benefits and avoid being overly technical. Be casual and pretend you are chatting with a friend. Avoid using several emojis. Be brief because this is a chatbot conversation. You should customize your presentation of the offer based on the specifics of the customer's situation. At the end include the payment link. When selecting a service, here are your priorties:
1. Avoid selling too much more than the customer needs 
2. Try to sell an ongoing plan whereever possible unless the customer says they don't want one
3. Try a more expensive plan first if you think the customer can afford it
Continue the conversation with the customer.
Your Analysis: {situation_analysis}
Conversation So Far: 
{conversation_history}
Services you can choose from: 
{service_offers_formatted}
Your pitch:
"""

parser_prompt = PromptTemplate(template=pitch_template, input_variables=["conversation_history", "service_offers_formatted", "situation_analysis"])
sales_pitch_chain = LLMChain(llm=chat, prompt=parser_prompt, output_key="pitch")

class Payments:
    def __init__(self, api_key):
        self.api_key = api_key
    
    def create_payment_link(self, service_id, initial):
        stripe.api_key = self.api_key
        unit_amount = int(initial * 100)
        #create the price for the service in the stripe dashboard
        price = stripe.Price.create(currency="usd", unit_amount=unit_amount, product=service_id)
        price_id = price.id
        #create the payment link
        link = stripe.checkout.Session.create(
            payment_method_types=['card'],
            line_items=[
                {
                    'price': price_id,
                    'quantity': 1,
                },
            ],
            mode='payment',
            success_url='https://example.com/success',
            cancel_url='https://example.com/cancel',
        )
        return link.url


pay = Payments(api_key=os.getenv('STRIPE_KEY'))

def add_payment_links(inputs):
    print('STEP: Getting payment links from stripe')
    services = inputs["complete_services"]
    updated_services = []
    for service in services:
        service_id = service.dict['Service ID']
        try:
            initial = float(service.dict['Initial Price'])
        except:
            initial = 0
            print('Error in creating payment link')
        if initial == 0:
            try:
                initial = float(service.dict['Recurring Price'])
            except:
                initial = 0
                print('Error in creating payment link')
        link = pay.create_payment_link(service_id, initial)
        service.dict['Payment Link'] = link
        updated_services.append(service)
    
    print('STEP: Payment links added to services')
    print('STEP: Creating pitch')
    return { "complete_services_with_links": updated_services }

payment_links_chain = TransformChain(
    transform=add_payment_links,
    input_variables=["complete_services"],
    output_variables=["complete_services_with_links"]
)

#add the payment link to the pitch
def parse_pitch(inputs):
    print(GREEN, 'STEP: Pitch complete' + RESET)
    print(GREEN, 'Adding payment link to pitch' + RESET)
    pitch = inputs["pitch"]
    #find the string in the form of <pay:service_name>
    payment_link = re.search(r'<pay:(.*?)>', pitch)
    if payment_link:
        print(GREEN, 'Found payment link' + RESET)
        program_name = payment_link.group(1).strip()
    else:
        print(RED, 'No payment link found' + RESET)
        program_name = ''
    #find the service that matches the name
    services = inputs["complete_services_with_links"]
    payment_link = "http://error.com"
    for service in services:
        if service.dict['Name'] == program_name:
            payment_link = service.dict['Payment Link']
    
    #replace the string with the payment link
    pitch = re.sub(r'<pay:(.*?)>', payment_link, pitch)
    print(GREEN, 'Payment link added' + RESET)
    return { "final pitch": pitch }

parse_pitch_chain = TransformChain(
    transform=parse_pitch,
    input_variables=["pitch", "complete_services"],
    output_variables=["final pitch"]
)

analyze_situation_sequential = SequentialChain(
    chains=[
        retrieve_situational_awareness,
        analyze_issue_chain,
        severity_grade_chain,
        get_severity_grade_chain,
        get_relevant_services_chain,
        calculate_prices_chain,
        payment_links_chain,
        sales_pitch_chain,
        parse_pitch_chain
    ],
    input_variables=["square_footage", "property_type", "acres", "target", "conversation_history"],
    output_variables=[
        "situational_experience", 
        "situation_analysis", 
        "relevant_services_dict",
        "complete_services",
        "service_offers_formatted",
        "pitch",
        "final pitch",
        "severity_grade",
        "severity_grade_raw"
    ],
    verbose=True
)

Running Chroma using direct local API.
loaded in 78 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection


In [4]:
result = analyze_situation_sequential({
    "square_footage": 1000,
    "property_type": "Single Family",
    "acres": 0,
    "target": "rats",
    "conversation_history": "rat problem"
})



> Entering new SequentialChain chain...
STEP: Getting situational experiences
Situational experiences found:  ['If a customer seems dishonest or self-contradictory in their description of their rodent issue or seems to be withholding information, the situation is probably worse than you think', 'If a customer seems dishonest or self-contradictory in their description of their rodent issue or seems to be withholding information, the situation is probably worse than you think', 'If a customer seems dishonest or self-contradictory in their description of their rodent issue or seems to be withholding information, the situation is probably worse than you think'] 
STEP: Analyzing situational experiences
STEP: Getting severity grade
Severity grade found:  medium 
STEP: Getting relevant services
Analysis:  Based on the limited information provided, it is difficult to determine the severity of the rat problem. However, since the property type is a single family home, it is possible that the i

In [5]:
for service in result['relevant_services_dict']:
    print(service.dict['Name'])
    print(service.dict['Initial Formula'])
    print(service.dict['Initial Price'])
    print(service.dict['Recurring Formula'])
    print(service.dict['Recurring Price'])

Program 8
if("medium"="high", 199, if("medium"="medium", 149, 0))
149
switch("rats", "ants", 75, "spiders", 75, "roaches", 90, "rats", 85, "mice", 85, "millipedes", 75, "centipedes", 75,  80) + if(1000 >= 4000, 10, if(1000 >= 2500, 5, 0))
85.0
Program 9
if("medium"="high", 250, if("medium"="medium", 225, 200))
225
if("medium"="high", 10, if("medium"="medium", 5, 0)) + if(1000 >= 4000, 8, if(1000 >= 2500, 4, 0)) + 75
80.0
Program 11
((1000/5)+1200) +if("medium" = "high", 300, 0)
1400.0
switch("rats", "ants", 75, "spiders", 75, "roaches", 90, "rats", 85, "mice", 85, "millipedes", 75, "centipedes", 75,  80) + if(1000 >= 4000, 10, if(1000 >= 2500, 5, 0)) + 50
135.0


In [6]:
result['pitch']

"Hey there! Based on the rat problem you mentioned earlier, I would recommend our Monthly Pest and Rodent Control program. It's a great ongoing rodent and pest control program that covers various pests, mice and rats. We come out monthly to do a perimeter treatment and set up and maintain a rodent baiting program to keep those pesky rodents at bay. It's perfect for customers who want year-round protection and peace of mind from those seasonal invaders. The initial price is $149, and it costs $85 per month thereafter. \n\nBut if you're facing a more severe rodent problem or want to ensure maximum protection against termites, I would suggest our Castle with Sentricon program. This bimonthly pest control plan covers most pests, proactive rodent issues, and even provides a Sentricon baiting system for curative or proactive termite solutions. It costs $1400 initially and $135 per month after that. It's a great option to reduce the risk of massive, surprise bills, as it provides year-round p

In [7]:
service_description_search.similarity_search('Test')

[Document(page_content='{"Name":"Program 6","Cost":"Average","Best for":"Customers who have a mosquito issue and want a green strategy or don\'t want spraying","Description":"Green tick and mosquito control program involving mosquito traps that are placed throughout the property. Offers a lower amount of protection than spraying.","Covered Pests":"mosquitoes","Level of Protection":"Average"}', lookup_str='', metadata={'source': 'data/services/Program 6.json'}, lookup_index=0),
 Document(page_content='{"Name":"Program 6","Cost":"Average","Best for":"Customers who have a mosquito issue and want a green strategy or don\'t want spraying","Description":"Green tick and mosquito control program involving mosquito traps that are placed throughout the property. Offers a lower amount of protection than spraying.","Covered Pests":"mosquitoes","Level of Protection":"Average"}', lookup_str='', metadata={'source': 'data/services/Program 6.json'}, lookup_index=0),
 Document(page_content='{"Name":"Pro